In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import scipy
import scipy.stats as stats
import statsmodels.api as sm
import pylab
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score
from math import sqrt

from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit

from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures

In [40]:
df = pd.read_excel('All.xlsx', sheet_name = 0)
df.head(5)

,LN(Цена),Цена,Город,ОПлощадь,Этаж_1,Этаж_2,Этаж_3,Этаж_4,Этажность_1,ГП,Ремонт_перв,Ремонт_бин,Долгота,Широта,Раст_центр_1,Раст_центр_2,S_инт,Объем
0,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,стандартный,с_ремонтом,60.51,56.81,5858.72,от_5000_до_10000,от_65_до_85,1154
1,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,стандартный,с_ремонтом,60.51,56.81,5858.72,от_5000_до_10000,от_65_до_85,1154
2,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,стандартный,с_ремонтом,60.51,56.81,5858.72,от_5000_до_10000,от_65_до_85,1154
3,15.78,7100000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,стандартный,с_ремонтом,60.51,56.81,5858.72,от_5000_до_10000,от_65_до_85,1154
4,15.76,7000000,Екатеринбург,70.00,3,средний,этажи_2-5,средний,26,2009,стандартный,с_ремонтом,60.51,56.81,5858.72,от_5000_до_10000,от_65_до_85,1154


In [41]:
enc = OneHotEncoder(handle_unknown = 'ignore')
enc_df = pd.DataFrame(enc.fit_transform(df[['Город', 'Этаж_3', 'Раст_центр_2']]).toarray())

enc_df.columns = enc.get_feature_names_out() #см ниже
#enc_df
df = df.join(enc_df)
df

,LN(Цена),Цена,Город,ОПлощадь,Этаж_1,Этаж_2,Этаж_3,Этаж_4,Этажность_1,ГП,...,Этаж_3_более_25,Этаж_3_первый,Этаж_3_последний,Этаж_3_этажи_11-25,Этаж_3_этажи_2-5,Этаж_3_этажи_6-10,Раст_центр_2_более_10000,Раст_центр_2_до_1500,Раст_центр_2_от_1500_до_5000,Раст_центр_2_от_5000_до_10000
0,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
1,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
2,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
3,15.78,7100000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
4,15.76,7000000,Екатеринбург,70.00,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34870,15.62,6100000,Краснодар,36.90,4,средний,этажи_2-5,средний,24,2022,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00
34871,15.61,5998000,Краснодар,40.00,17,последний,последний,первый_последний,17,2023,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34872,15.52,5500000,Краснодар,36.60,17,средний,этажи_11-25,средний,24,2022,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00
34873,15.50,5369000,Краснодар,36.60,20,средний,этажи_11-25,средний,24,2022,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00


In [43]:
encoder = LabelEncoder()
df['Ремонт_бин_enc'] = encoder.fit_transform(df[['Ремонт_бин']])
df

C:\Users\skapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,LN(Цена),Цена,Город,ОПлощадь,Этаж_1,Этаж_2,Этаж_3,Этаж_4,Этажность_1,ГП,...,Этаж_3_первый,Этаж_3_последний,Этаж_3_этажи_11-25,Этаж_3_этажи_2-5,Этаж_3_этажи_6-10,Раст_центр_2_более_10000,Раст_центр_2_до_1500,Раст_центр_2_от_1500_до_5000,Раст_центр_2_от_5000_до_10000,Ремонт_бин_enc
0,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
1,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
2,15.76,7000000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
3,15.78,7100000,Екатеринбург,70.30,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
4,15.76,7000000,Екатеринбург,70.00,3,средний,этажи_2-5,средний,26,2009,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34870,15.62,6100000,Краснодар,36.90,4,средний,этажи_2-5,средний,24,2022,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1
34871,15.61,5998000,Краснодар,40.00,17,последний,последний,первый_последний,17,2023,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1
34872,15.52,5500000,Краснодар,36.60,17,средний,этажи_11-25,средний,24,2022,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1
34873,15.50,5369000,Краснодар,36.60,20,средний,этажи_11-25,средний,24,2022,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1


In [44]:
OHE_name = enc.get_feature_names_out()
OHE_name

array(['Город_Владивосток', 'Город_Екатеринбург', 'Город_Казань',
       'Город_Краснодар', 'Этаж_3_более_25', 'Этаж_3_первый',
       'Этаж_3_последний', 'Этаж_3_этажи_11-25', 'Этаж_3_этажи_2-5',
       'Этаж_3_этажи_6-10', 'Раст_центр_2_более_10000',
       'Раст_центр_2_до_1500', 'Раст_центр_2_от_1500_до_5000',
       'Раст_центр_2_от_5000_до_10000'], dtype=object)

In [45]:
features_num = ['ОПлощадь', 'Этажность_1', 'ГП', 'Долгота', 'Широта', 'Объем']
features_all = features_num + list(OHE_name) + ['Ремонт_бин_enc'] # собираем числовые и OHE факторы в один список

X = df[features_all] # создаем наш итоговый набор факторов
X

,ОПлощадь,Этажность_1,ГП,Долгота,Широта,Объем,Город_Владивосток,Город_Екатеринбург,Город_Казань,Город_Краснодар,...,Этаж_3_первый,Этаж_3_последний,Этаж_3_этажи_11-25,Этаж_3_этажи_2-5,Этаж_3_этажи_6-10,Раст_центр_2_более_10000,Раст_центр_2_до_1500,Раст_центр_2_от_1500_до_5000,Раст_центр_2_от_5000_до_10000,Ремонт_бин_enc
0,70.30,26,2009,60.51,56.81,1154,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
1,70.30,26,2009,60.51,56.81,1154,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
2,70.30,26,2009,60.51,56.81,1154,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
3,70.30,26,2009,60.51,56.81,1154,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
4,70.00,26,2009,60.51,56.81,1154,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34870,36.90,24,2022,39.04,45.08,3851,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1
34871,40.00,17,2023,39.04,45.08,3851,0.00,0.00,0.00,1.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1
34872,36.60,24,2022,39.04,45.08,3851,0.00,0.00,0.00,1.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1
34873,36.60,24,2022,39.04,45.08,3851,0.00,0.00,0.00,1.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1


In [46]:
# Создаем полиномиальные признаки
degree = 2

poly = PolynomialFeatures(degree = degree, include_bias = False) #include_bias = False - создаем степени (degree = 2) и парные произведения
X_poly = poly.fit_transform(X)

X_poly

array([[7.030e+01, 2.600e+01, 2.009e+03, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [7.030e+01, 2.600e+01, 2.009e+03, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [7.030e+01, 2.600e+01, 2.009e+03, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.660e+01, 2.400e+01, 2.022e+03, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [3.660e+01, 2.400e+01, 2.022e+03, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [3.670e+01, 2.400e+01, 2.022e+03, ..., 1.000e+00, 1.000e+00,
        1.000e+00]])

In [47]:
X_poly = pd.DataFrame(X_poly, columns = poly.get_feature_names_out(X.columns))
X_poly

,ОПлощадь,Этажность_1,ГП,Долгота,Широта,Объем,Город_Владивосток,Город_Екатеринбург,Город_Казань,Город_Краснодар,...,Раст_центр_2_до_1500^2,Раст_центр_2_до_1500 Раст_центр_2_от_1500_до_5000,Раст_центр_2_до_1500 Раст_центр_2_от_5000_до_10000,Раст_центр_2_до_1500 Ремонт_бин_enc,Раст_центр_2_от_1500_до_5000^2,Раст_центр_2_от_1500_до_5000 Раст_центр_2_от_5000_до_10000,Раст_центр_2_от_1500_до_5000 Ремонт_бин_enc,Раст_центр_2_от_5000_до_10000^2,Раст_центр_2_от_5000_до_10000 Ремонт_бин_enc,Ремонт_бин_enc^2
0,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
1,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
2,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
3,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,70.00,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34870,36.90,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
34871,40.00,17.00,2023.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
34872,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
34873,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00


### Удалим OHE факторы со степенью 2

In [48]:
OHE_name_all = list(OHE_name) + ['Ремонт_бин_enc'] 

OHE_name_del = []

for feature_name in OHE_name_all:
    square_feature_name = f"{feature_name}^2"
    if square_feature_name in X_poly.columns:
        OHE_name_del.append(square_feature_name)

OHE_name_del

['Город_Владивосток^2',
 'Город_Екатеринбург^2',
 'Город_Казань^2',
 'Город_Краснодар^2',
 'Этаж_3_более_25^2',
 'Этаж_3_первый^2',
 'Этаж_3_последний^2',
 'Этаж_3_этажи_11-25^2',
 'Этаж_3_этажи_2-5^2',
 'Этаж_3_этажи_6-10^2',
 'Раст_центр_2_более_10000^2',
 'Раст_центр_2_до_1500^2',
 'Раст_центр_2_от_1500_до_5000^2',
 'Раст_центр_2_от_5000_до_10000^2',
 'Ремонт_бин_enc^2']

In [49]:
X_poly = X_poly.drop(OHE_name_del, axis = 1)
X_poly

,ОПлощадь,Этажность_1,ГП,Долгота,Широта,Объем,Город_Владивосток,Город_Екатеринбург,Город_Казань,Город_Краснодар,...,Раст_центр_2_более_10000 Раст_центр_2_до_1500,Раст_центр_2_более_10000 Раст_центр_2_от_1500_до_5000,Раст_центр_2_более_10000 Раст_центр_2_от_5000_до_10000,Раст_центр_2_более_10000 Ремонт_бин_enc,Раст_центр_2_до_1500 Раст_центр_2_от_1500_до_5000,Раст_центр_2_до_1500 Раст_центр_2_от_5000_до_10000,Раст_центр_2_до_1500 Ремонт_бин_enc,Раст_центр_2_от_1500_до_5000 Раст_центр_2_от_5000_до_10000,Раст_центр_2_от_1500_до_5000 Ремонт_бин_enc,Раст_центр_2_от_5000_до_10000 Ремонт_бин_enc
0,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,70.00,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34870,36.90,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34871,40.00,17.00,2023.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34872,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34873,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00


### Удалим парные произведения OHE факторов в рамках одной категории (нулевые столбцы)

Мы знаем, что приозведение категориальных факторов в рамках одной категории дает на выходе нулевой столбец.

In [51]:
cols_to_drop = (X_poly.columns[X_poly.eq(0).all()])
cols_to_drop #[0:60]

Index(['Город_Владивосток Город_Екатеринбург',
       'Город_Владивосток Город_Казань', 'Город_Владивосток Город_Краснодар',
       'Город_Екатеринбург Город_Казань', 'Город_Екатеринбург Город_Краснодар',
       'Город_Казань Город_Краснодар', 'Город_Краснодар Этаж_3_более_25',
       'Этаж_3_более_25 Этаж_3_первый', 'Этаж_3_более_25 Этаж_3_последний',
       'Этаж_3_более_25 Этаж_3_этажи_11-25',
       'Этаж_3_более_25 Этаж_3_этажи_2-5', 'Этаж_3_более_25 Этаж_3_этажи_6-10',
       'Этаж_3_более_25 Раст_центр_2_более_10000',
       'Этаж_3_первый Этаж_3_последний', 'Этаж_3_первый Этаж_3_этажи_11-25',
       'Этаж_3_первый Этаж_3_этажи_2-5', 'Этаж_3_первый Этаж_3_этажи_6-10',
       'Этаж_3_последний Этаж_3_этажи_11-25',
       'Этаж_3_последний Этаж_3_этажи_2-5',
       'Этаж_3_последний Этаж_3_этажи_6-10',
       'Этаж_3_этажи_11-25 Этаж_3_этажи_2-5',
       'Этаж_3_этажи_11-25 Этаж_3_этажи_6-10',
       'Этаж_3_этажи_2-5 Этаж_3_этажи_6-10',
       'Раст_центр_2_более_10000 Раст_центр

In [52]:
X_poly = X_poly.drop(cols_to_drop, axis = 1)
X_poly

,ОПлощадь,Этажность_1,ГП,Долгота,Широта,Объем,Город_Владивосток,Город_Екатеринбург,Город_Казань,Город_Краснодар,...,Этаж_3_этажи_2-5 Ремонт_бин_enc,Этаж_3_этажи_6-10 Раст_центр_2_более_10000,Этаж_3_этажи_6-10 Раст_центр_2_до_1500,Этаж_3_этажи_6-10 Раст_центр_2_от_1500_до_5000,Этаж_3_этажи_6-10 Раст_центр_2_от_5000_до_10000,Этаж_3_этажи_6-10 Ремонт_бин_enc,Раст_центр_2_более_10000 Ремонт_бин_enc,Раст_центр_2_до_1500 Ремонт_бин_enc,Раст_центр_2_от_1500_до_5000 Ремонт_бин_enc,Раст_центр_2_от_5000_до_10000 Ремонт_бин_enc
0,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,70.00,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34870,36.90,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34871,40.00,17.00,2023.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34872,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34873,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00


### Удалим парные произведения OHE факторов в рамках одной категории по порогу

In [55]:
features_cat = df[['Город', 'Этаж_3', 'Раст_центр_2', 'Ремонт_бин']]
features_cat

,Город,Этаж_3,Раст_центр_2,Ремонт_бин
0,Екатеринбург,этажи_2-5,от_5000_до_10000,с_ремонтом
1,Екатеринбург,этажи_2-5,от_5000_до_10000,с_ремонтом
2,Екатеринбург,этажи_2-5,от_5000_до_10000,с_ремонтом
3,Екатеринбург,этажи_2-5,от_5000_до_10000,с_ремонтом
4,Екатеринбург,этажи_2-5,от_5000_до_10000,с_ремонтом
...,...,...,...,...
34870,Краснодар,этажи_2-5,от_5000_до_10000,требует_ремонта
34871,Краснодар,последний,от_5000_до_10000,требует_ремонта
34872,Краснодар,этажи_11-25,от_5000_до_10000,требует_ремонта
34873,Краснодар,этажи_11-25,от_5000_до_10000,требует_ремонта


In [56]:
for col in features_cat.columns:
    print(features_cat[col].value_counts())
    print('==========================================')

Город
Краснодар       12911
Казань           9499
Екатеринбург     7967
Владивосток      4498
Name: count, dtype: int64
Этаж_3
этажи_2-5      12060
этажи_6-10      8184
этажи_11-25     7400
последний       3909
первый          3245
более_25          77
Name: count, dtype: int64
Раст_центр_2
от_5000_до_10000    18273
от_1500_до_5000     11797
более_10000          3890
до_1500               915
Name: count, dtype: int64
Ремонт_бин
с_ремонтом         23918
требует_ремонта    10957
Name: count, dtype: int64


По табличкам выше можно сделать вывод, что в рамках одной категории наблюдается много объектов за исключением 77 и 915.

In [58]:
pd.options.display.float_format = '{:.2f}'.format 

column_sums = X_poly.sum()
column_sums.sort_values()[0:60]

Город_Казань Этаж_3_более_25                         2.00
Город_Владивосток Этаж_3_более_25                    6.00
Этаж_3_более_25 Раст_центр_2_до_1500                 7.00
Этаж_3_более_25 Ремонт_бин_enc                      17.00
Этаж_3_более_25 Раст_центр_2_от_5000_до_10000       32.00
Этаж_3_более_25 Раст_центр_2_от_1500_до_5000        38.00
Город_Екатеринбург Этаж_3_более_25                  69.00
Этаж_3_первый Раст_центр_2_до_1500                  72.00
Этаж_3_более_25                                     77.00
Город_Казань Раст_центр_2_до_1500                   99.00
Этаж_3_последний Раст_центр_2_до_1500              124.00
Этаж_3_этажи_11-25 Раст_центр_2_до_1500            164.00
Этаж_3_этажи_6-10 Раст_центр_2_до_1500             182.00
Город_Владивосток Раст_центр_2_до_1500             224.00
Город_Екатеринбург Раст_центр_2_до_1500            225.00
Раст_центр_2_до_1500 Ремонт_бин_enc                260.00
Этаж_3_этажи_2-5 Раст_центр_2_до_1500              366.00
Город_Краснода

Можно удалить те факторы, по которым меньше 1000 или 3000 объектов. Возьмем порог 2000.

In [59]:
threshold = 2000

cols_to_drop = column_sums[column_sums <= threshold].index
cols_to_drop

Index(['Этаж_3_более_25', 'Раст_центр_2_до_1500',
       'Город_Владивосток Этаж_3_более_25', 'Город_Владивосток Этаж_3_первый',
       'Город_Владивосток Этаж_3_последний',
       'Город_Владивосток Этаж_3_этажи_11-25',
       'Город_Владивосток Этаж_3_этажи_2-5',
       'Город_Владивосток Этаж_3_этажи_6-10',
       'Город_Владивосток Раст_центр_2_более_10000',
       'Город_Владивосток Раст_центр_2_до_1500',
       'Город_Владивосток Раст_центр_2_от_1500_до_5000',
       'Город_Владивосток Ремонт_бин_enc',
       'Город_Екатеринбург Этаж_3_более_25',
       'Город_Екатеринбург Этаж_3_первый',
       'Город_Екатеринбург Этаж_3_последний',
       'Город_Екатеринбург Этаж_3_этажи_11-25',
       'Город_Екатеринбург Этаж_3_этажи_6-10',
       'Город_Екатеринбург Раст_центр_2_более_10000',
       'Город_Екатеринбург Раст_центр_2_до_1500',
       'Город_Екатеринбург Ремонт_бин_enc', 'Город_Казань Этаж_3_более_25',
       'Город_Казань Этаж_3_первый', 'Город_Казань Этаж_3_последний',
       

In [60]:
X_poly = X_poly.drop(cols_to_drop, axis = 1)
X_poly

,ОПлощадь,Этажность_1,ГП,Долгота,Широта,Объем,Город_Владивосток,Город_Екатеринбург,Город_Казань,Город_Краснодар,...,Этаж_3_этажи_11-25 Раст_центр_2_от_5000_до_10000,Этаж_3_этажи_11-25 Ремонт_бин_enc,Этаж_3_этажи_2-5 Раст_центр_2_от_1500_до_5000,Этаж_3_этажи_2-5 Раст_центр_2_от_5000_до_10000,Этаж_3_этажи_2-5 Ремонт_бин_enc,Этаж_3_этажи_6-10 Раст_центр_2_от_1500_до_5000,Этаж_3_этажи_6-10 Раст_центр_2_от_5000_до_10000,Этаж_3_этажи_6-10 Ремонт_бин_enc,Раст_центр_2_от_1500_до_5000 Ремонт_бин_enc,Раст_центр_2_от_5000_до_10000 Ремонт_бин_enc
0,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
1,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
2,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
3,70.30,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
4,70.00,26.00,2009.00,60.51,56.81,1154.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34870,36.90,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00
34871,40.00,17.00,2023.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34872,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
34873,36.60,24.00,2022.00,39.04,45.08,3851.00,0.00,0.00,0.00,1.00,...,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00


Итого, с 252 факторов сократились до 156.